### TODO

- Correct it, it's finding local mimima
- Find the right way to turn the solution into integers

In [52]:
import numpy as np
import scipy.optimize as optimize

In [266]:
def init_simulation(num_nodes, 
                    num_days, 
                    icu_capacities = None, 
                    transport_capacities = None, 
                    ini_path = None, 
                    demand_min = 1, 
                    demand_max = 5,
                    icu_min = 10,
                    icu_max = 15,
                    transport_min = 2,
                    transport_max = 10):
    if ini_path:
        print("you can also upload an .ini file")
        raise NotImplemented
    else:
        demand = []
        for node in range(num_nodes):
            demand.append(list(np.random.randint(demand_min, demand_max, size = num_days)))
        
        icu_capacities = list(np.random.randint(icu_min, icu_max, size = num_nodes)) if not icu_capacities else icu_capacities
        transport_capacities = list(np.random.randint(transport_min, transport_max, size = num_nodes)) if not transport_capacities else transport_capacities        
    
    movements_orig = np.array([[[0]*num_nodes]*num_nodes]*num_days)
    return icu_capacities, transport_capacities, demand, movements_orig

In [267]:
prob_death_transport = 0
num_days = 30
num_nodes = 5
node_capacities, transport_capacities, demands, movements_orig = init_simulation(num_nodes, num_days)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

movements = movements_orig.ravel()

Randomly generated
30 days, 5 hospitals
ICU capacities of hospitals: [11, 10, 13, 14, 14]
Daily transport capacities: [2, 7, 8, 4, 8], 
Daily simulated demand (from uniform distrution): 
[[3, 2, 3, 4, 2, 3, 2, 1, 3, 3, 3, 2, 4, 4, 2, 4, 3, 2, 2, 3, 3, 3, 4, 4, 2, 1, 3, 3, 3, 1], [3, 2, 1, 1, 4, 3, 3, 3, 2, 1, 4, 3, 1, 3, 1, 1, 3, 1, 3, 3, 2, 1, 2, 3, 1, 1, 3, 4, 2, 4], [2, 1, 3, 2, 2, 2, 2, 2, 3, 3, 4, 1, 3, 3, 1, 3, 2, 3, 2, 2, 2, 1, 2, 2, 3, 3, 2, 1, 2, 2], [1, 2, 1, 1, 3, 4, 2, 4, 4, 4, 3, 2, 3, 2, 2, 1, 4, 2, 3, 4, 1, 4, 3, 3, 1, 1, 1, 2, 3, 2], [1, 3, 4, 4, 3, 1, 1, 1, 2, 3, 1, 3, 3, 4, 4, 4, 4, 2, 1, 4, 2, 1, 2, 1, 4, 2, 4, 1, 1, 2]]


In [268]:
def calculate_outgoing(array,day,node):
    return sum(array[day][node])

def calculate_incoming(array,day,node):
    total_outgoing = 0
    for n in range(num_nodes):
        total_outgoing = total_outgoing + array[day][n][node]
    return total_outgoing


def demand_day_node_raw(node_capacity, demand_day, left_day, received_day, demand_previous_days, left_previous_days, received_previous_days):
    return demand_day - left_day + received_day+ min(demand_previous_days-left_previous_days+received_previous_days, node_capacity)


def demand_day_node(movements,day,node):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    total_received_patients = 0
    total_outbound_patients = 0
    accumulated_demand = 0
    for d in range(day):
        total_received_patients = total_received_patients + calculate_incoming(movements_res,d,node)
        total_outbound_patients = total_outbound_patients + calculate_outgoing(movements_res,d,node)
    accumulated_demand = sum(demands[node][:day])
    
    demand = max(0, demand_day_node_raw(node_capacities[node],demands[node][day], 
                                        calculate_outgoing(movements_res,day,node), calculate_incoming(movements_res,day,node),
                                        accumulated_demand, total_outbound_patients, total_received_patients                
                ))
    return demand

def calc_total_deaths(movements):
    total = 0
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    for node in range(num_nodes):
        for day in range(num_days):
            deaths_not_attended = max(0, demand_day_node(movements,day,node) - node_capacities[node])
            deaths_transport = prob_death_transport*movements_res[day].sum()
            total = total + deaths_not_attended + deaths_transport
#             print(f'Node: {node}, Day: {day}, Deaths: {deaths_not_attended}, Total: {total}')
    return total

def outgoing_list(movements):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    outgoing_list = list()
    for day in range(num_days):
        for node in range(num_nodes):
            outgoing_list.append(calculate_outgoing(movements_res,day,node))
    return outgoing_list

def generate_bounds():
    outgoing_list = list()
    for day in range(num_days):
        for outgoing_node in range(num_nodes):
            for incoming_node in range(num_nodes):
                outgoing_list.append((0,transport_capacities[outgoing_node]))
    return outgoing_list

def f_cons(node_id, day):
    return lambda x: transport_capacities[node_id] - calculate_outgoing(np.reshape(x,(num_days,num_nodes,num_nodes)),day ,node_id)


In [269]:
cons = []
for node in range(num_nodes):
    for day in range(num_days):
        cons.append({'type': 'ineq', 'fun': f_cons(node,day)})

In [270]:
initial_guess = movements.copy()
bounds_movement = generate_bounds()
result = optimize.minimize(calc_total_deaths, initial_guess, method='SLSQP', constraints=cons, bounds=bounds_movement)

In [271]:
np.reshape(np.rint(result.x),(num_days,num_nodes,num_nodes))

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]

In [272]:
calc_total_deaths(np.rint(result.x))

301.0

In [273]:
calc_total_deaths(movements)

301

In [274]:
for node in range(num_nodes):
    for day in range(num_days):
        print(f'Node: {node}, Day: {day}, Demand: {demand_day_node(np.rint(result.x),day,node)}')

Node: 0, Day: 0, Demand: 3.0
Node: 0, Day: 1, Demand: 5.0
Node: 0, Day: 2, Demand: 8.0
Node: 0, Day: 3, Demand: 12.0
Node: 0, Day: 4, Demand: 13.0
Node: 0, Day: 5, Demand: 14.0
Node: 0, Day: 6, Demand: 13.0
Node: 0, Day: 7, Demand: 12.0
Node: 0, Day: 8, Demand: 14.0
Node: 0, Day: 9, Demand: 14.0
Node: 0, Day: 10, Demand: 14.0
Node: 0, Day: 11, Demand: 13.0
Node: 0, Day: 12, Demand: 15.0
Node: 0, Day: 13, Demand: 15.0
Node: 0, Day: 14, Demand: 13.0
Node: 0, Day: 15, Demand: 15.0
Node: 0, Day: 16, Demand: 14.0
Node: 0, Day: 17, Demand: 13.0
Node: 0, Day: 18, Demand: 13.0
Node: 0, Day: 19, Demand: 14.0
Node: 0, Day: 20, Demand: 14.0
Node: 0, Day: 21, Demand: 14.0
Node: 0, Day: 22, Demand: 15.0
Node: 0, Day: 23, Demand: 15.0
Node: 0, Day: 24, Demand: 13.0
Node: 0, Day: 25, Demand: 12.0
Node: 0, Day: 26, Demand: 14.0
Node: 0, Day: 27, Demand: 14.0
Node: 0, Day: 28, Demand: 14.0
Node: 0, Day: 29, Demand: 12.0
Node: 1, Day: 0, Demand: 3.0
Node: 1, Day: 1, Demand: 5.0
Node: 1, Day: 2, Demand: 

In [275]:
demand_day_node(np.rint(result.x),3,2)

8.0

# SANDBOX

The old fixed example

In [174]:
node_capacities = [10,5,7]
transport_capacities = [4,1,1]
demands = [[2,4,5,3],[0,1,1,1],[0,3,2,0]]
movements_orig = np.array(
                  [[[0,0,0],
                   [0,0,0],
                   [0,0,0]],
                   
                  [[0,0,0],
                   [0,0,0],
                   [0,0,0]],
                   
                  [[0,0,0],
                   [0,0,0],
                   [0,0,0]],
                   
                  [[0,0,0],
                   [0,0,0],
                   [0,0,0]]
                 ])
prob_death_transport = 0
movements = movements_orig.ravel()
num_days = len(demands[0])
num_nodes = len(node_capacities)
movements_orig.shape

(4, 3, 3)